In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog, Button, Label, Entry, messagebox
from PIL import Image, ImageTk

class SteganographyApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Steganography")
        self.root.geometry("500x400")
        
        # Variables
        self.input_image_path = ""
        self.output_image_path = ""
        self.secret_message = ""
        
        # GUI Elements
        Label(root, text="Image Steganography", font=("Helvetica", 16)).pack(pady=10)
        
        Button(root, text="Select Image", command=self.select_image).pack(pady=5)
        self.image_label = Label(root, text="No image selected")
        self.image_label.pack(pady=5)
        
        Label(root, text="Secret Message:").pack(pady=5)
        self.message_entry = Entry(root, width=50)
        self.message_entry.pack(pady=5)
        
        Button(root, text="Encode Message", command=self.encode_message).pack(pady=5)
        Button(root, text="Decode Message", command=self.decode_message).pack(pady=5)
        Button(root, text="Save Output Image", command=self.save_image).pack(pady=5)
        
        self.result_label = Label(root, text="", wraplength=450)
        self.result_label.pack(pady=10)
    
    def select_image(self):
        self.input_image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp")])
        if self.input_image_path:
            self.image_label.config(text=f"Selected: {self.input_image_path.split('/')[-1]}")
    
    def save_image(self):
        if hasattr(self, 'encoded_image'):
            self.output_image_path = filedialog.asksaveasfilename(defaultextension=".png", 
                                                                 filetypes=[("PNG files", "*.png")])
            if self.output_image_path:
                cv2.imwrite(self.output_image_path, self.encoded_image)
                messagebox.showinfo("Success", "Image saved successfully!")
    
    def encode_message(self):
        if not self.input_image_path:
            messagebox.showerror("Error", "Please select an image first")
            return
            
        self.secret_message = self.message_entry.get()
        if not self.secret_message:
            messagebox.showerror("Error", "Please enter a secret message")
            return
            
        try:
            # Read the image
            image = cv2.imread(self.input_image_path)
            if image is None:
                raise ValueError("Could not read the image")
                
            # Convert message to binary
            binary_message = ''.join([format(ord(i), '08b') for i in self.secret_message])
            binary_message += '1111111111111110'  # Adding a delimiter
            
            if len(binary_message) > image.size * 3:
                messagebox.showerror("Error", "Message too large for the selected image")
                return
                
            # Embed the message
            self.encoded_image = self._embed_message(image.copy(), binary_message)
            
            # Display the encoded image
            self._display_image(self.encoded_image)
            self.result_label.config(text="Message encoded successfully!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Encoding failed: {str(e)}")
    
    def _embed_message(self, image, binary_message):
        data_index = 0
        for row in image:
            for pixel in row:
                for color_channel in range(3):  # R, G, B
                    if data_index < len(binary_message):
                        # Modify the least significant bit
                        pixel[color_channel] = (pixel[color_channel] & 254) | int(binary_message[data_index])
                        data_index += 1
        return image
    
    def decode_message(self):
        if not self.input_image_path:
            messagebox.showerror("Error", "Please select an image first")
            return
            
        try:
            # Read the image
            image = cv2.imread(self.input_image_path)
            if image is None:
                raise ValueError("Could not read the image")
                
            # Extract binary data
            binary_data = ""
            for row in image:
                for pixel in row:
                    for color_channel in range(3):  # R, G, B
                        binary_data += str(pixel[color_channel] & 1)
                        
            # Find the delimiter
            delimiter = '1111111111111110'
            if delimiter in binary_data:
                binary_data = binary_data[:binary_data.index(delimiter)]
                
            # Convert binary to text
            message = ""
            for i in range(0, len(binary_data), 8):
                byte = binary_data[i:i+8]
                if len(byte) == 8:
                    message += chr(int(byte, 2))
                    
            if message:
                self.result_label.config(text=f"Decoded message: {message}")
            else:
                self.result_label.config(text="No message found or decoding failed")
                
        except Exception as e:
            messagebox.showerror("Error", f"Decoding failed: {str(e)}")
    
    def _display_image(self, image):
        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Resize for display if too large
        height, width = image_rgb.shape[:2]
        max_size = 400
        if height > max_size or width > max_size:
            scale = max_size / max(height, width)
            image_rgb = cv2.resize(image_rgb, (int(width * scale), int(height * scale)))
        
        # Convert to PhotoImage
        img = Image.fromarray(image_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        
        # Update the label
        self.image_label.config(image=imgtk)
        self.image_label.image = imgtk

if __name__ == "__main__":
    root = Tk()
    app = SteganographyApp(root)
    root.mainloop()